In [4]:
from iminuit import Minuit
import numpy as np
import math
from math import*
from matplotlib import pyplot as plt
import pandas as pd
import statistics


In [5]:
#=====================================================================================================================================
#required functions:
def f1(t): #fit func: a0_fit*exp(-b*x) + a1_fit*exp{-(x-mu_fit)**2/a_fit**2}, fit values obtained from fitting data with fit func.
    a0_fit = 7.072927906619347 
    b_fit = -1.1814512107411528 
    a1_fit = 82.63120365913976
    mu_fit = 125.3
    a_fit = 2
    return t**(a0_fit + b_fit*math.log(t)) + (a1_fit*math.e**(-(t - mu_fit)**2/a_fit**2))
def f2(t): #fit func: a0_fit*exp(-b*x), fit values obtained from fitting data with fit func.
    a0_fit = 7.0716143377141725 
    b_fit = -1.181045308790491
    return t**(a0_fit + b_fit*math.log(t))
#=======================================================================================================================================
#functions to create and store toy datasets:    
def num1(): #create toy data by introducing Poisson fluctuation in the bkgd fit model
    data = np.loadtxt('cms_mass_events_2012_bkg_w.txt')
    x = (data[:,0])
    r = len(x)
    y=[]
    for i in range(r):
        y.append(f2(x[i]))
    n = np.random.poisson(lam=(y), size=(34)) 
    print("num1",n)
    print("num1",x)
    np.savetxt('toy_data_bkg_local_f.txt',n)
    #print(n)
    return n
def num2(): #store the toy data
    n=[]
    dat = np.loadtxt('toy_data_bkg_local_f.txt')
    for i in dat:
        n.append(i)
    #print("num2",n)  
    return n
def num3(): #create toy data by introducing Poisson fluctuation in the data fit model
    data = np.loadtxt('cms_mass_events_2012_bkg_w.txt')
    x = (data[:,0])
    r = len(x)
    y=[]
    for i in range(r):
        y.append(f1(x[i]))
    n = np.random.poisson(lam=(y), size=(34))
    np.savetxt('toy_data_sig_local_f.txt',n)
    #print(n)
    return n
def num4(): #store the toy data
    n=[]
    dat = np.loadtxt('toy_data_sig_local_f.txt')
    for i in dat:
        n.append(i)
    #print(n)  
    return n         
#=====================================================================================================================================    
#neg-log-likelihood functions:
def fact(n):
    f=1
    for i in range(1,n+1):
        f = f*i
    return f    
    
def nll1(a0,b): #neg-log-likelihood func for toy data generated from bkgd only fit with and exp
    data = np.loadtxt('cms_mass_events_2012_bkg_w.txt')
    x = (data[:,0])
    r = len(x)
    n = num2() 
    f=0       
    #i>14 to i<=24 sig-reg
    for i in range(r): 
        f_i = x[i]**(a0 + b*math.log(x[i]))
        n_i = math.floor(n[i])
        if(f_i<=0):
           print("f_i",f_i)
        f = f + (f_i - n[i]*math.log(f_i) + n_i*math.log(n_i) - n_i)
    return f

def nll2(a0,b,a1): #neg-log-likelihood func for toy data generated from bkgd only fit with and exp+gauss
    data = np.loadtxt('cms_mass_events_2012_bkg_w.txt')
    x = (data[:,0])
    r = len(x)
    n = num2()    
    f=0     
    mu = 125.3
    a = 2
    #i>14 to i<=24 sig-reg
    for i in range(r): 
           n_i = math.floor(n[i])
           f_i = x[i]**(a0 + b*math.log(x[i])) + (a1*math.e**(-(x[i] - mu)**2/a**2))
           if(f_i<=0):
              print("f_i",f_i)
           f = f + (f_i - n[i]*math.log(f_i) + n_i*math.log(n_i) - n_i)
    return f

def nll5(a0,b): #neg-log-likelihood func for toy data generated from bkgd+sig fit with and exp
    data = np.loadtxt('cms_mass_events_2012_bkg_w.txt')
    x = (data[:,0])
    r = len(x)
    r1 = len(x)
    n = num4() 
    f=0       
    #i>14 to i<=24 sig-reg
    for i in range(r): 
        f_i = x[i]**(a0 + b*math.log(x[i]))
        n_i = math.floor(n[i])
        if(f_i<=0):
          print("f_i",f_i)
        f = f + (f_i - n[i]*math.log(f_i) + n_i*math.log(n_i) - n_i)
    return f

def nll6(a0,b,a1): #neg-log-likelihood func for toy data generated from bkgd+sig fit with and exp+gauss
    data = np.loadtxt('cms_mass_events_2012_bkg_w.txt')
    x = (data[:,0])
    r = len(x)
    n = num4()    
    f=0     
    mu = 125.3
    a = 2
    #i>14 to i<=24 sig-reg
    for i in range(r):
           n_i = math.floor(n[i])
           f_i = x[i]**(a0 + b*math.log(x[i])) + (a1*math.e**(-(x[i] - mu)**2/a**2))
           if(f_i<=0):
              print("f_i",f_i)
           f = f + (f_i - n[i]*math.log(f_i) + n_i*math.log(n_i) - n_i)
    return f

def nll3(n,a0,b): #calculates neg-log-likelihood and chisq for exp fit
    lchi=[]
    chisq=0
    data = np.loadtxt('cms_mass_events_2012_bkg_w.txt')
    x = (data[:,0])
    r = len(x) 
    f=0       
    #i>14 to i<=24 sig-reg
    #print(n)
    for i in range(r): 
           n_i = math.floor(n[i])
           f_i = x[i]**(a0 + b*math.log(x[i]))
           chisq_i = (f_i - n[i])**2/(f_i)
           chisq += chisq_i
           f = f + (f_i - n[i]*math.log(f_i) + n_i*math.log(n_i) - n_i)
    lchi.append(f)
    lchi.append(chisq)       
    return lchi

def nll4(n,a0,b,a1): #calculates neg-log-likelihood and chisq for exp+gauss fit
    lchi=[]
    chisq=0
    data = np.loadtxt('cms_mass_events_2012_bkg_w.txt')
    x = (data[:,0])
    r = len(x) 
    f=0     
    mu = 125.3
    a = 2
    #i>14 to i<=24 sig-reg
    for i in range(r): 
           n_i = math.floor(n[i])
           f_i = x[i]**(a0 + b*math.log(x[i])) + (a1*math.e**(-(x[i] - mu)**2/a**2))
           f = f + (f_i - n[i]*math.log(f_i) + n_i*math.log(n_i) - n_i)
           chisq_i = (f_i - n[i])**2/(f_i)
           chisq += chisq_i
    lchi.append(f)
    lchi.append(chisq)       
    return lchi


In [6]:
#===============================================================(main code)====================================================================    
llrb=[]
chi1=[]
chi2=[]
fit_a0_exp=[]
fit_b_exp=[]
neg_like_exp=[]
fit_a0_gxp=[]
fit_b_gxp=[]
fit_a1=[]
neg_like_gxp=[]
nll_diff=[]
a1_neg=[]
like_ratio=[]
k_exp=0
ka_exp=0
k_gxp=0
ka_gxp=0
neg_a1=0
pos_a1=0
neg_nll=0
k=0
for i in range(10000000):
    gxp_valid = False
    gxp_acc = False
    #print('=====================================================')
    n=num3()  #put num1() for bkgd, num3() for full data
    m=Minuit(nll5,a0=2,b=0) #use nll1 for bkgd, nll5 for full data
    #m.limits["a0"] = (0,100000)
    #m.limits["b"] = (0,1)
    m.migrad()
    m.hesse()
    print('exp-valid-acc',m.valid,m.accurate)
    if(m.valid):
      k_exp=k_exp+1    
    if(m.accurate):
       ka_exp=ka_exp+1
    a0_fit_exp = m.values["a0"]
    b_fit_exp = m.values["b"]
    print("exp-fit",a0_fit_exp,b_fit_exp)
    
    
    m=Minuit(nll6,a0=2,b=0,a1=70)  #use nll2 for bkgd, nll6 for full data
    #m.limits["a0"] = (0,100000)
    m.limits["a1"] = (0,1000)
    #m.limits["b"] = (0,1)
    m.migrad()
    m.hesse()
    print('gxp-valid-acc',m.valid,m.accurate)
    if(m.valid):
      k_gxp=k_gxp+1   
      gxp_valid = True 
    if(m.accurate):
       ka_gxp=ka_gxp+1
       gxp_acc = True
    a0_fit_gxp = m.values["a0"]
    b_fit_gxp = m.values["b"]
    a1_fit_gxp = m.values["a1"]
    print("exp-gauss-fit",a0_fit_gxp,b_fit_gxp,a1_fit_gxp)  
    
    if (gxp_valid*gxp_acc):
       k=k+1
       
       lchi1 = nll3(n,a0_fit_exp,b_fit_exp)
       nl1 = lchi1[0]
       chi1.append(lchi1[1]) 
       fit_a0_exp.append(a0_fit_exp)
       fit_b_exp.append(b_fit_exp)
       neg_like_exp.append(nl1)
       
       lchi2 = nll4(n,a0_fit_gxp,b_fit_gxp,a1_fit_gxp)
       nl2 = lchi2[0]
       chi2.append(lchi2[1]) 
       fit_a0_gxp.append(a0_fit_gxp)
       fit_b_gxp.append(b_fit_gxp)
       fit_a1.append(a1_fit_gxp)
       neg_like_gxp.append(nl2)
       nll_diff.append(2*(nl1-nl2))
       
       llrb.append(2*(nl1-nl2))
       
       if ((a1_fit_gxp)<0.001):
          neg_a1 = neg_a1+1
       if ((nl1-nl2)<0):
          neg_nll = neg_nll+1
          print(nl1-nl2,math.e**(nl2-nl1),a1_fit_gxp)
          like_ratio.append(math.e**(nl2-nl1))
          a1_neg.append(a1_fit_gxp)
       
       #print("iteration ", i, "neg log-like exp ", nl1, "  neg log-like exp+gauss ", nl2, "  neg log ratio, exp/exp+gauss ", llrb[k-1])
       
    if (k>=1000):
       break   
   
print('nll-diff-neg-pos',neg_a1,neg_nll)
data = {'a0_exp': fit_a0_exp,'b_exp': fit_b_exp,'nll_exp': neg_like_exp, 'a0_gxp': fit_a0_gxp, 'b_gxp': fit_b_gxp, 'a1_gxp': fit_a1,
        'nll_gxp': neg_like_gxp, 'nll_diff': nll_diff}
dat = {'likelihood ratio': like_ratio, 'a1_gxp': a1_neg}        
df=pd.DataFrame(data)
df1=pd.DataFrame(dat)
#print(df) 
print('significance',sqrt(statistics.mean(nll_diff)))
#print("bkgd: valid acc exp ",k_exp,ka_exp, " bkgd: valid acc gxp ", k_gxp,ka_gxp)
df.to_csv('fit_val_cms_2012_calib_f2.csv') 
df1.to_csv('neg-nll-like-a1_f2.csv')   
np.savetxt("nll-ratio-10K-stir-full-cms_2012_calib_f2.txt",llrb)
np.savetxt("chisq-exp-10K-stir-full-cms_2012_calib_f2.txt",chi1)
np.savetxt("chisq-gxp-10K-stir-full-cms_2012_calib_f2.txt",chi2)

'''print("===============================================full===================================================")    
llrsb=[]
kf_exp=0
kaf_exp=0
kf_gxp=0
kaf_gxp=0
for i in range(10000):
    n=num3()
    m=Minuit(nll5,a0=10000,b=0.03)
    m.limits["a0"] = (0,5000000)
    m.limits["b"] = (0,1)
    m.migrad()
    m.hesse()
    if(Minuit.valid):
      kf_exp=kf_exp+1    
    if(Minuit.accurate):
       kaf_exp=kaf_exp+1
    a0_fit = m.values["a0"]
    b_fit = m.values["b"]
    llrs1 = nll3(n,a0_fit,b_fit)

    m=Minuit(nll6,a0=10000,b=0.03,a1=10)
    m.limits["a0"] = (0,50000000)
    m.limits["a1"] = (0,10000)
    m.limits["b"] = (0,1)
    m.migrad()
    m.hesse()
    if(Minuit.valid):
      kf_gxp=kf_gxp+1    
    if(Minuit.accurate):
       kaf_gxp=kaf_gxp+1
    a0_fit = m.values["a0"]
    b_fit = m.values["b"]
    a1_fit = m.values["a1"]
    llrs2 = nll4(n,a0_fit,b_fit,a1_fit)
    
    llrsb.append(llrs1/llrs2)
    print("iteration ", i, "neg log-like exp ", llrs1, "  neg log-like exp+gauss ", llrs2, "  neg log ratio, exp/exp+gauss ",llrsb[i])  
np.savetxt("nll-ratio-full-data.txt",llrsb)  
#print("full: valid acc exp ",kf_exp,kaf_exp, " full: valid acc gxp ", kf_gxp,kaf_gxp) ''' 
#======================================================================================================================================                


exp-valid-acc True True
exp-fit 7.063190069944066 -1.178691949172516
gxp-valid-acc True True
exp-gauss-fit 7.073528915354756 -1.1812039611443792 85.98910564283345
exp-valid-acc True True
exp-fit 7.217318083097113 -1.2109778885815492
gxp-valid-acc True True
exp-gauss-fit 7.231689076682171 -1.2144110486071804 104.01367796342213
exp-valid-acc True True
exp-fit 6.994295735267746 -1.1647287584100268
gxp-valid-acc True True
exp-gauss-fit 7.00500610613299 -1.167349561053928 92.61360697309448
exp-valid-acc True True
exp-fit 7.060319923614076 -1.178580725300142
gxp-valid-acc True True
exp-gauss-fit 7.071207743653969 -1.1812149166512382 86.8046232499127
exp-valid-acc True True
exp-fit 7.085289934334964 -1.1839949010411281
gxp-valid-acc True True
exp-gauss-fit 7.09261189992067 -1.1857587088968278 56.512382977597454
exp-valid-acc True True
exp-fit 7.13633392124837 -1.1940870328944508
gxp-valid-acc True True
exp-gauss-fit 7.153060017139449 -1.1981152715722898 129.29730866468282
exp-valid-acc True T

/tmp/ipykernel_912/3235835662.py:121: RuntimeWarning: overflow encountered in double_scalars
  f_i = x[i]**(a0 + b*math.log(x[i])) + (a1*math.e**(-(x[i] - mu)**2/a**2))
/tmp/ipykernel_912/3235835662.py:124: RuntimeWarning: invalid value encountered in double_scalars
  f = f + (f_i - n[i]*math.log(f_i) + n_i*math.log(n_i) - n_i)


gxp-valid-acc True True
exp-gauss-fit 7.055936170349249 -1.1782110082679376 68.22326852552054
exp-valid-acc True True
exp-fit 7.069742150986569 -1.1796247098098347
gxp-valid-acc True True
exp-gauss-fit 7.082891926313534 -1.182845215978005 116.22364699302899
exp-valid-acc True True
exp-fit 7.058570654652233 -1.1778642074480143
gxp-valid-acc True True
exp-gauss-fit 7.066746237719079 -1.179861240725495 70.06383559171523
exp-valid-acc True True
exp-fit 6.985086979512567 -1.16283603961308
gxp-valid-acc True True
exp-gauss-fit 6.9998699649867415 -1.1664672312443791 130.79931347049924
exp-valid-acc True True
exp-fit 7.199607071705292 -1.2077215786204785
gxp-valid-acc True True
exp-gauss-fit 7.210200318568685 -1.2102626448495404 78.68950060318713
exp-valid-acc True True
exp-fit 7.081162191424672 -1.1823300720784307
gxp-valid-acc True True
exp-gauss-fit 7.0931730015273935 -1.1852492557663499 100.2006790085016
exp-valid-acc True True
exp-fit 6.917686594086398 -1.1480341963682656
gxp-valid-acc Tr

exp-valid-acc True True
exp-fit 7.068278707052264 -1.180083454684899
gxp-valid-acc True True
exp-gauss-fit 7.07806309936042 -1.1824694364329877 82.06518709324605
exp-valid-acc True True
exp-fit 7.02307101920467 -1.170903092776417
gxp-valid-acc True True
exp-gauss-fit 7.0359347090360025 -1.174040543397429 108.09338891995701
exp-valid-acc True True
exp-fit 6.993206218629148 -1.1646479909646796
gxp-valid-acc True True
exp-gauss-fit 7.001162475735179 -1.1666027072740726 70.35041114677797
exp-valid-acc True True
exp-fit 7.177482114271172 -1.203076577141006
gxp-valid-acc True True
exp-gauss-fit 7.1870326188431495 -1.2053808450335786 74.03910973438701
exp-valid-acc True True
exp-fit 7.118496172827083 -1.1906435944138147
gxp-valid-acc True True
exp-gauss-fit 7.129632128217299 -1.1933321541527644 87.48101317814073
exp-valid-acc True True
exp-fit 7.020362597577446 -1.1696988191591127
gxp-valid-acc True True
exp-gauss-fit 7.027226285904298 -1.1713871497953232 62.17796681994723
exp-valid-acc True 

gxp-valid-acc True True
exp-gauss-fit 7.069146787444191 -1.1801023790543752 45.741318274650425
exp-valid-acc True True
exp-fit 7.165103935610538 -1.199685721773204
gxp-valid-acc True True
exp-gauss-fit 7.182191379301238 -1.2037942486988504 131.87137519195062
exp-valid-acc True True
exp-fit 7.0292139909293345 -1.1724197737437598
gxp-valid-acc True True
exp-gauss-fit 7.040060348400307 -1.1750627580658197 90.06621390525446
exp-valid-acc True True
exp-fit 7.03181840367497 -1.1726981749081813
gxp-valid-acc True True
exp-gauss-fit 7.039396607140283 -1.1745491775045598 64.44278002169332
exp-valid-acc True True
exp-fit 6.968948787781268 -1.159167566101715
gxp-valid-acc True True
exp-gauss-fit 6.974822070017313 -1.160614949879884 53.62464866123229
exp-valid-acc True True
exp-fit 7.019734556646352 -1.1697410965557995
gxp-valid-acc True True
exp-gauss-fit 7.030612714659096 -1.172409053857816 96.16047803837678
exp-valid-acc True True
exp-fit 7.112834012047601 -1.1896899744058935
gxp-valid-acc True

...

gxp-valid-acc True True
exp-gauss-fit 7.107528325565207 -1.1886811926881395 58.13242706118682
exp-valid-acc True True
exp-fit 7.091391219333004 -1.1849572522213334
gxp-valid-acc True True
exp-gauss-fit 7.105468424226679 -1.1883591509413054 111.22265412528758
exp-valid-acc True True
exp-fit 7.081360070142317 -1.1826662987577998
gxp-valid-acc True True
exp-gauss-fit 7.086700784557442 -1.1839652436049972 43.995114580127094
exp-valid-acc True True
exp-fit 7.105119263493474 -1.1879644025557263
gxp-valid-acc True True
exp-gauss-fit 7.114569209316801 -1.1902492661013326 74.69179621350303
exp-valid-acc True True
exp-fit 7.0075220162415555 -1.1672916330658625
gxp-valid-acc True True
exp-gauss-fit 7.016591978549028 -1.169525954294453 81.94114606505273
exp-valid-acc True True
exp-fit 7.176304437732151 -1.2029319138551149
gxp-valid-acc True True
exp-gauss-fit 7.1865043283912655 -1.2053677361674893 73.61917322983365
exp-valid-acc True True
exp-fit 6.9935205177643915 -1.1643409607132393
gxp-valid-ac

exp-valid-acc True True
exp-fit 7.015252246893116 -1.1693854536763395
gxp-valid-acc True True
exp-gauss-fit 7.023657619354692 -1.1714429638338213 72.31398640564213
exp-valid-acc True True
exp-fit 7.020597846358977 -1.1693894106131855
gxp-valid-acc True True
exp-gauss-fit 7.031308957987827 -1.1720032301688437 92.7660680893437
exp-valid-acc True True
exp-fit 7.060747403594849 -1.178683610666278
gxp-valid-acc True True
exp-gauss-fit 7.0732530354194525 -1.1817206714925195 102.27094058171825
exp-valid-acc True True
exp-fit 7.066127956238242 -1.1797398977663376
gxp-valid-acc True True
exp-gauss-fit 7.074499057397667 -1.181776406147524 69.45670068458269
exp-valid-acc True True
exp-fit 7.107101002321145 -1.1885233120686547
gxp-valid-acc True True
exp-gauss-fit 7.11561329394704 -1.1905828091898685 67.85156914793804
exp-valid-acc True True
exp-fit 7.113897452509021 -1.1899682683658694
gxp-valid-acc True True
exp-gauss-fit 7.120991544273956 -1.1916779155598842 54.53803022671908
exp-valid-acc True

gxp-valid-acc True True
exp-gauss-fit 7.1444901627302615 -1.1963166128537555 117.25252517802433
exp-valid-acc True True
exp-fit 7.0577451623951175 -1.1775370761078408
gxp-valid-acc True True
exp-gauss-fit 7.061612109135085 -1.1784873655507384 34.615940590276566
exp-valid-acc True True
exp-fit 7.0048559639417185 -1.1667376281253499
gxp-valid-acc True True
exp-gauss-fit 7.014611140965734 -1.1691272638348276 85.19820939239645
exp-valid-acc True True
exp-fit 7.022690180012632 -1.1706600408544505
gxp-valid-acc True True
exp-gauss-fit 7.031019608849749 -1.1726962077772514 70.88634908096196
exp-valid-acc True True
exp-fit 7.000448746730919 -1.1658267843466892
gxp-valid-acc True True
exp-gauss-fit 7.010432635636545 -1.16828570858158 89.261467942238
exp-valid-acc True True
exp-fit 7.148137679559237 -1.196818261291839
gxp-valid-acc True True
exp-gauss-fit 7.1546500302873675 -1.198384039564576 49.66356379595308
exp-valid-acc True True
exp-fit 7.017410358370003 -1.1698304225453282
gxp-valid-acc Tr

exp-valid-acc True True
exp-fit 7.000196550733037 -1.1659013766609339
gxp-valid-acc True True
exp-gauss-fit 7.007921538482114 -1.1678000657600536 69.00624108234153
exp-valid-acc True True
exp-fit 7.040320357518258 -1.1745634884125118
gxp-valid-acc True True
exp-gauss-fit 7.055169817537319 -1.1781877743520939 125.2693523159385
exp-valid-acc True True
exp-fit 7.143389526801842 -1.1960656569005508
gxp-valid-acc True True
exp-gauss-fit 7.151565674104701 -1.1980297256005872 61.38234285945539
exp-valid-acc True True
exp-fit 7.029312954269553 -1.171876415013035
gxp-valid-acc True True
exp-gauss-fit 7.044892655559638 -1.1756634029182715 129.5623445527253
exp-valid-acc True True
exp-fit 7.002548140419391 -1.1662062503067232
gxp-valid-acc True True
exp-gauss-fit 7.013154632606555 -1.1688130744463117 94.75981775774288
exp-valid-acc True True
exp-fit 7.078701062500954 -1.182447726093627
gxp-valid-acc True True
exp-gauss-fit 7.08634695117971 -1.1843069554496914 63.018213749408375
exp-valid-acc True

'print("===============================================full===================================================")    \nllrsb=[]\nkf_exp=0\nkaf_exp=0\nkf_gxp=0\nkaf_gxp=0\nfor i in range(10000):\n    n=num3()\n    m=Minuit(nll5,a0=10000,b=0.03)\n    m.limits["a0"] = (0,5000000)\n    m.limits["b"] = (0,1)\n    m.migrad()\n    m.hesse()\n    if(Minuit.valid):\n      kf_exp=kf_exp+1    \n    if(Minuit.accurate):\n       kaf_exp=kaf_exp+1\n    a0_fit = m.values["a0"]\n    b_fit = m.values["b"]\n    llrs1 = nll3(n,a0_fit,b_fit)\n\n    m=Minuit(nll6,a0=10000,b=0.03,a1=10)\n    m.limits["a0"] = (0,50000000)\n    m.limits["a1"] = (0,10000)\n    m.limits["b"] = (0,1)\n    m.migrad()\n    m.hesse()\n    if(Minuit.valid):\n      kf_gxp=kf_gxp+1    \n    if(Minuit.accurate):\n       kaf_gxp=kaf_gxp+1\n    a0_fit = m.values["a0"]\n    b_fit = m.values["b"]\n    a1_fit = m.values["a1"]\n    llrs2 = nll4(n,a0_fit,b_fit,a1_fit)\n    \n    llrsb.append(llrs1/llrs2)\n    print("iteration ", i, "neg log-li